# Exp 5. This is a notebook using GraphQAChain with Knowledge Triples from the graph to communicate with the database

Ref: https://python.langchain.com/en/latest/modules/chains/index_examples/graph_qa.html

In [1]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
from langchain.chains import GraphQAChain
from langchain.graphs.networkx_graph import KnowledgeTriple


In [2]:
load_dotenv()
neo4j_url = os.getenv('NEO4J_URL')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')
openai_key = os.getenv('OPENAI_KEY')
pinecone_api_key = os.getenv('PINECONE_KEY')

os.environ['OPENAI_API_KEY'] = openai_key

In [3]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [4]:
# Enquiry neo4j graph database
def get_pair_nodes (tx):
    pairs_of_nodes = []
    for record in tx.run("MATCH (a)-[r]->(b) RETURN labels(a), a.name, type(r), labels(b), b.name"):
        pair_node = {
            "start_node_name": record["a.name"],
            "start_node_label":record["labels(a)"][0],
            "end_node_name": record["b.name"],
            "end_node_label":record["labels(b)"][0],
            "edge": record["type(r)"]
        }
        pairs_of_nodes.append(pair_node)
    return pairs_of_nodes

driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))
with driver.session() as session:
    pairs_of_nodes = session.execute_read(get_pair_nodes)
driver.close()

pairs_of_nodes[:3]

[{'start_node_name': 'Review_project_portfolio_balance_for_2020_Q1',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'Portfolio_management',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Conduct_project_assurance_readiness_for_Brexit_Programme',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Identify_best_practice_regulatory_project_as_examplar',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'}]

In [5]:
# Build text base
text = ""
for p_n in pairs_of_nodes:
    text_string = f"'{p_n['start_node_name']}' {p_n['edge']} '{p_n['end_node_name']}'.\n"
    text += text_string

In [6]:
text


"'Review_project_portfolio_balance_for_2020_Q1' deploy 'Portfolio_management'.\n'Conduct_project_assurance_readiness_for_Brexit_Programme' deploy 'P3M_Assessment'.\n'Identify_best_practice_regulatory_project_as_examplar' deploy 'P3M_Assessment'.\n'Identify_best_practice_regulatory_project_as_examplar' deploy 'P3M_Support'.\n'Cross-project_review_of_plastic_dependencies' deploy 'P3M_Optimisation'.\n'Rescope_large_project_which_has_failed_to_deliver_in_2020' deploy 'Project_Programme_recovery'.\n'Rescope_large_project_which_has_failed_to_deliver_in_2020' deploy 'P3M_Assessment'.\n'Poor_project_execution' influences 'Non-Delivery_of_benefits'.\n'Poor_project_execution' influences 'Business_disruption_'.\n'Executive_sponsorship' mitigates 'Non-Delivery_of_benefits'.\n'Steering_group' mitigates 'Slowing_digitisation_pace'.\n'Regular_progress_updates' mitigates 'Poor_project_execution'.\n'Broad_stakeholder_representation' mitigates 'Slowing_digitisation_pace'.\n'Link_with_digital_programme_t

In [7]:
graph = index_creator.from_text(text)

In [8]:
print(graph.get_triples(),len(graph.get_triples()))

[('Review_project_portfolio_balance_for_2020_Q1', 'Portfolio_management', 'deploys'), ('Conduct_project_assurance_readiness_for_Brexit_Programme', 'P3M_Assessment', 'deploys'), ('Identify_best_practice_regulatory_project_as_examplar', 'P3M_Assessment', 'deploys'), ('Identify_best_practice_regulatory_project_as_examplar', 'P3M_Support', 'deploys'), ('Cross-project_review_of_plastic_dependencies', 'P3M_Optimisation', 'deploys'), ('Rescope_large_project_which_has_failed_to_deliver_in_2020', 'Project_Programme_recovery', 'deploys'), ('Rescope_large_project_which_has_failed_to_deliver_in_2020', 'P3M_Assessment', 'deploys'), ('Poor_project_execution', 'Non-Delivery_of_benefits', 'influences'), ('Poor_project_execution', 'Business', 'influences')] 9


<font size=4 color="red">The triples function with index creator from text ONLY output partial information resulting in misleading and missing answer

In [9]:
chain = GraphQAChain.from_llm(OpenAI(openai_api_key=openai_key, temperature=0), graph=graph, verbose=True)


In [10]:
chain.run("What types of values that Unilever_create?")




> Entering new GraphQAChain chain...
Entities Extracted:
 Unilever
Full Context:



Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.



> Finished chain.


'\n\nUnilever creates values related to sustainability, health and well-being, and environmental protection.'

In [11]:
chain.run("What are the project risks")



> Entering new GraphQAChain chain...
Entities Extracted:
 NONE
Full Context:


> Finished chain.


" I don't know."

# Add triples rather than simply from text

Ref: https://apex974.com/articles/explore-langchain-support-for-knowledge-graph

In [12]:
for p_n in pairs_of_nodes:
    graph.add_triple(KnowledgeTriple(p_n['start_node_name'], p_n['end_node_name'], p_n['edge']))

In [13]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)
chain.run("What types of values that Unilever_create?")




> Entering new GraphQAChain chain...
Entities Extracted:
 Unilever
Full Context:


> Finished chain.


'\n\nUnilever creates values related to sustainability, health and well-being, and environmental protection.'

In [14]:
chain.run("What are the advantages of Operating with Purpose")



> Entering new GraphQAChain chain...
Entities Extracted:
 NONE
Full Context:


> Finished chain.


'\n\nOperating with purpose has many advantages, including increased motivation, improved focus, and a greater sense of meaning and fulfillment. It can also help to create a more positive work environment, as employees are more likely to be engaged and productive when they feel their work has purpose. Additionally, operating with purpose can help to create a more successful business, as employees are more likely to be productive and creative when they are working towards a meaningful goal.'

In [12]:
graph.write_to_gml("ps-graph.gml")
